In [1]:
import io
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

2021-10-11 00:29:01.321371: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import os  
import cv2
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

# Loading np array from images
def make_dataset(labels, dir_path, IMG_SIZE = 150):
    # initial an empty list X to store image of np.array()
    X = []

    # initial an empty list Z to store labels/names of cat individauls
    Z = []
    for label in labels:
        DIR = dir_path + label
        for img in tqdm(os.listdir(DIR)):
            path = os.path.join(DIR,img)
            # reading images
            img = cv2.imread(path,cv2.IMREAD_COLOR)
            # resizing images to (150, 150, 3), 3 is the number of channels - RGB
            img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
            X.append(np.array(img))
            Z.append(str(label)) 
    ## Transform labels in Z to Y from class 0 to class 9, as 10 different cat individuals
    le=LabelEncoder()
    Y=le.fit_transform(Z)

    ## Transform and normalize X in the range of [0, 1]
    X=np.array(X)
    X=X/255.
    return X, Y

In [3]:
dir_path = '/kaggle/input/squirrel-monkey/Saimiri sciureus/'
IMG_SIZE = 150
labels = [str(i) for i in range(44)]

X, Y = make_dataset(labels, dir_path)

100%|██████████| 4/4 [00:00<00:00, 198.21it/s]


In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test ,y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 2021)

In [5]:
imsize = 150
lr = 1e-4
EPOCHS = 20
batch_size = 256

base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                      input_shape = (imsize,imsize,3),
                      weights = 'imagenet')

# freeze all the layers of VGG, so they won't be trained.
for layer in base_model_16.layers:
    layer.trainable = False

model_vgg_16 = tf.keras.models.Sequential([
    base_model_16,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.Dense(units=44, activation='softmax')
])

model_vgg_16.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='sparse_categorical_crossentropy', metrics=['acc']) 

history = model_vgg_16.fit(x=x_train, y=y_train,
                          epochs=EPOCHS,
                          batch_size=batch_size,
                          verbose=1)

2021-10-11 00:29:09.564244: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-11 00:29:09.567102: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-10-11 00:29:09.605560: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-11 00:29:09.606180: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-10-11 00:29:09.606234: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-10-11 00:29:09.632258: I tensorflow/stream_executor/platform/def

58892288/58889256 [==============================] - 1s 0us/step


2021-10-11 00:29:13.879299: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-11 00:29:13.889282: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000175000 Hz


Epoch 1/20


2021-10-11 00:29:14.626338: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-10-11 00:29:15.388584: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-10-11 00:29:15.425005: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


1/1 [==============================] - 14s 14s/step - loss: 3.9770 - acc: 0.0348
Epoch 2/20
1/1 [==============================] - 0s 207ms/step - loss: 3.5324 - acc: 0.1045
Epoch 3/20
1/1 [==============================] - 0s 206ms/step - loss: 3.2233 - acc: 0.3134
Epoch 4/20
1/1 [==============================] - 0s 206ms/step - loss: 3.0004 - acc: 0.3333
Epoch 5/20
1/1 [==============================] - 0s 192ms/step - loss: 2.8054 - acc: 0.3632
Epoch 6/20
1/1 [==============================] - 0s 190ms/step - loss: 2.6065 - acc: 0.4527
Epoch 7/20
1/1 [==============================] - 0s 187ms/step - loss: 2.4094 - acc: 0.5124
Epoch 8/20
1/1 [==============================] - 0s 203ms/step - loss: 2.2228 - acc: 0.5721
Epoch 9/20
1/1 [==============================] - 0s 204ms/step - loss: 2.0469 - acc: 0.6070
Epoch 10/20
1/1 [==============================] - 0s 205ms/step - loss: 1.8820 - acc: 0.6716
Epoch 11/20
1/1 [==============================] - 0s 194ms/step - loss: 1.7296 -

In [6]:
model_vgg_16.evaluate(x_test, y_test)[1]

2/2 [==============================] - 3s 939ms/step - loss: 1.8638 - acc: 0.6078


0.6078431606292725

In [7]:
y_pred = model_vgg_16.predict_classes(x_test)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


# Table of misclassifying classes

In [8]:
labels, counts = np.unique(y_test, return_counts = True)

ind = (y_test != y_pred)
mis_class= y_test[ind]
labels_mis, counts_mis = np.unique(mis_class, return_counts = True)

cnt = []
for i in labels_mis:
    cnt.append(counts[np.where(labels == i)][0])
cnt = np.array(cnt)

mis_rate = counts_mis/cnt

import pandas as pd
pd.DataFrame({'miss_classes': labels_mis, 
              'miss_counts': counts_mis,
              'total_counts': cnt,
              'miss_rate': mis_rate})

,miss_classes,miss_counts,total_counts,miss_rate
0,5,3,3,1.0
1,6,1,2,0.5
2,9,1,2,0.5
3,10,1,1,1.0
4,11,1,1,1.0
5,14,2,2,1.0
6,19,1,1,1.0
7,25,1,1,1.0
8,26,1,1,1.0
9,27,2,2,1.0


# Contrastive Loss

In [9]:
# Helper Function 

# Create positive pairs and negative pairs
import random
def create_pairs(images, labels):
    numClasses = 44
    # initialize two empty lists to hold the (image, image) pairs and
    # labels to indicate if a pair is positive (0) or negative (1)
    np.random.seed(2021)
    pairImages = []
    pairLabels = []
    
    # calculate the total number of classes present in the dataset
    # and then build a list of indexes for each class label that
    # provides the indexes for all examples with a given label
    idx = [np.where(labels == i)[0] for i in range(44)]
    
    # loop voer all images
    for idxA in range(len(images)):
        # grab the current image and label belonging to the current iteration
        currentImage = images[idxA]
        label = labels[idxA]
        
        # randomly pick on an image that belongs to the *same* class label
        posId = random.choice(idx[label])
        posImage = images[posId]
        
        # prepare a positive pair and update the images and labels
        pairImages.append([currentImage, posImage])
        pairLabels.append([0])
        
        # grab the indices for each of the class labels *not* equal to
        # the current label and randomly pick an image corresponding
        # to a label *not* equal to the current label
        negId = np.where(labels != label)[0]
        negIdx = random.choice(negId)
        negImage = images[negIdx]
        
        # prepare a negative pair of images and update out lists
        pairImages.append([currentImage, negImage])
        pairLabels.append([1])
    
    return (np.array(pairImages), np.array(pairLabels))



# Function to calculate the distance between two images (Euclidean Distance used here)
import tensorflow.keras.backend as K
def euclidean_distance(vectors):
    # unpack the vectors into separate lists
    (featsA, featsB) = vectors
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1,
                       keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))


# contrastive loss function
def contrastive_loss(y, preds, margin=1):
    # explicitly cast the true class label data type to the predicted
    # class label data type (otherwise we run the risk of having two
    # separate data types, causing TensorFlow to error out)
    y = tf.cast(y, preds.dtype)
    # calculate the contrastive loss between the true labels and
    # the predicted labels
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean((1 - y) * squaredPreds + y * squaredMargin)
    # return the computed contrastive loss to the calling function
    return loss

(pairTrain, labelTrain) = create_pairs(x_train, y_train)

In [10]:
lr = 1e-4
embeddingDim = 128


base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                  input_shape = (imsize,imsize,3),
                  weights = 'imagenet')

# freeze all the layers of VGG, so they won't be trained.
for layer in base_model_16.layers:
    layer.trainable = False

model_cl = tf.keras.models.Sequential([
    base_model_16,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
])
    
imgA = tf.keras.layers.Input(shape=((imsize, imsize, 3)))
imgB = tf.keras.layers.Input(shape=((imsize, imsize, 3)))
    
featsA = model_cl(imgA)
featsB = model_cl(imgB)
   
distance = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])
model = tf.keras.Model(inputs=[imgA, imgB], outputs=distance)
model.compile(loss=contrastive_loss, optimizer=tf.keras.optimizers.Adam(lr))

history = model.fit([pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:],
                    batch_size = batch_size, 
                    epochs=EPOCHS, 
                    verbose=1)

Epoch 1/20
2/2 [==============================] - 30s 12s/step - loss: 0.1936
Epoch 2/20
2/2 [==============================] - 1s 288ms/step - loss: 0.1512
Epoch 3/20
2/2 [==============================] - 1s 310ms/step - loss: 0.1250
Epoch 4/20
2/2 [==============================] - 1s 298ms/step - loss: 0.1160
Epoch 5/20
2/2 [==============================] - 1s 310ms/step - loss: 0.0932
Epoch 6/20
2/2 [==============================] - 1s 284ms/step - loss: 0.0838
Epoch 7/20
2/2 [==============================] - 1s 309ms/step - loss: 0.0750
Epoch 8/20
2/2 [==============================] - 1s 359ms/step - loss: 0.0624
Epoch 9/20
2/2 [==============================] - 1s 295ms/step - loss: 0.0581
Epoch 10/20
2/2 [==============================] - 1s 284ms/step - loss: 0.0501
Epoch 11/20
2/2 [==============================] - 1s 283ms/step - loss: 0.0449
Epoch 12/20
2/2 [==============================] - 1s 280ms/step - loss: 0.0418
Epoch 13/20
2/2 [==============================] -

In [11]:
from sklearn.neighbors import KNeighborsClassifier

def acc_cl(x_test, y_test, k = 1):
    embedding_train_cl = []
    for i in range(len(y_train)):
        embedding_train_cl.append(model_cl(x_train[i].reshape(1, imsize, imsize, 3))[0])
    embedding_train_cl = np.array(embedding_train_cl, dtype=float) 

    knn_cl = KNeighborsClassifier(n_neighbors = k)
    knn_cl.fit(embedding_train_cl, y_train)
    
    x_test_embedding = model_cl(x_test)
    acc_close = round(knn_cl.score(x_test_embedding, y_test), 2)
    print('The accuracy on the Closed Dataset with contrastive loss is {}'.format(acc_close))
    
    return acc_close

In [12]:
acc_cl(x_test, y_test, k = 1) 

The accuracy on the Closed Dataset with contrastive loss is 0.71


0.71

In [13]:
def predict_cl(x_test, k = 1):
    embedding_train_cl = []
    for i in range(len(y_train)):
        embedding_train_cl.append(model_cl(x_train[i].reshape(1, imsize, imsize, 3))[0])
    embedding_train_cl = np.array(embedding_train_cl, dtype=float) 

    knn_cl = KNeighborsClassifier(n_neighbors = k)
    knn_cl.fit(embedding_train_cl, y_train)
    
    x_test_embedding = model_cl(x_test)
    pred =  knn_cl.predict(x_test_embedding)
    return pred

In [14]:
y_pred = predict_cl(x_test, k = 1)

# Table of misclassifying classes

In [15]:
labels, counts = np.unique(y_test, return_counts = True)

ind = (y_test != y_pred)
mis_class= y_test[ind]
labels_mis, counts_mis = np.unique(mis_class, return_counts = True)

cnt = []
for i in labels_mis:
    cnt.append(counts[np.where(labels == i)][0])
cnt = np.array(cnt)

mis_rate = counts_mis/cnt

import pandas as pd
pd.DataFrame({'miss_classes': labels_mis, 
              'miss_counts': counts_mis,
              'total_counts': cnt,
              'miss_rate': mis_rate})

,miss_classes,miss_counts,total_counts,miss_rate
0,5,3,3,1.0
1,11,1,1,1.0
2,14,1,2,0.5
3,19,1,1,1.0
4,25,1,1,1.0
5,26,1,1,1.0
6,27,2,2,1.0
7,31,1,2,0.5
8,37,2,2,1.0
9,40,1,1,1.0


# Triplet loss

In [16]:
# the loss is decreased fast when LR is between 5e-4 and 7e-4
# choose the mid point between 8e-5 and 4e-4
lr = 1e-3

base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                      input_shape = (imsize, imsize, 3),
                      weights = 'imagenet')

# freeze all the layers of VGG, so they won't be trained.
for layer in base_model_16.layers:
    layer.trainable = False

model_tl = tf.keras.models.Sequential([
    base_model_16,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
])

model_tl.compile(
    optimizer=tf.keras.optimizers.Adam(lr),
    loss=tfa.losses.TripletSemiHardLoss())

history = model_tl.fit(x=x_train, y= y_train,
                       batch_size=batch_size,
                       epochs=EPOCHS, 
                       verbose=1)

Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.9815
Epoch 2/20
1/1 [==============================] - 0s 198ms/step - loss: 0.9028
Epoch 3/20
1/1 [==============================] - 0s 211ms/step - loss: 0.7942
Epoch 4/20
1/1 [==============================] - 0s 212ms/step - loss: 0.6856
Epoch 5/20
1/1 [==============================] - 0s 210ms/step - loss: 0.6271
Epoch 6/20
1/1 [==============================] - 0s 210ms/step - loss: 0.4756
Epoch 7/20
1/1 [==============================] - 0s 196ms/step - loss: 0.4392
Epoch 8/20
1/1 [==============================] - 0s 210ms/step - loss: 0.3360
Epoch 9/20
1/1 [==============================] - 0s 195ms/step - loss: 0.2813
Epoch 10/20
1/1 [==============================] - 0s 212ms/step - loss: 0.2363
Epoch 11/20
1/1 [==============================] - 0s 211ms/step - loss: 0.1934
Epoch 12/20
1/1 [==============================] - 0s 195ms/step - loss: 0.1661
Epoch 13/20
1/1 [==============================] - 0

In [17]:
def acc_tl(x_test, y_test, k = 1):
    embedding_train_tl = []
    for i in range(len(y_train)):
        embedding_train_tl.append(model_tl(x_train[i].reshape(1, imsize, imsize, 3))[0])
        
    embedding_train_tl = np.array(embedding_train_tl, dtype=float) 

    knn_tl = KNeighborsClassifier(n_neighbors = k)
    knn_tl.fit(embedding_train_tl, y_train)
    
    x_test_embedding = model_tl(x_test)
    acc_close = round(knn_tl.score(x_test_embedding, y_test), 2)
    print('The accuracy on the Closed Dataset with triplet loss is {}'.format(acc_close))
    
    return acc_close

In [18]:
acc_tl(x_test, y_test, k = 1)

The accuracy on the Closed Dataset with triplet loss is 0.78


0.78

In [19]:
def predict_tl(x_test, k = 1):
    embedding_train_tl = []
    for i in range(len(y_train)):
        embedding_train_tl.append(model_tl(x_train[i].reshape(1, imsize, imsize, 3))[0])
        
    embedding_train_tl = np.array(embedding_train_tl, dtype=float) 

    knn_tl = KNeighborsClassifier(n_neighbors = k)
    knn_tl.fit(embedding_train_tl, y_train)
    
    x_test_embedding = model_tl(x_test)
    pred = knn_tl.predict(x_test_embedding)
    
    return pred

In [20]:
y_pred = predict_tl(x_test, k = 1)

# Tabel of misclassifying classes

In [21]:
labels, counts = np.unique(y_test, return_counts = True)

ind = (y_test != y_pred)
mis_class= y_test[ind]
labels_mis, counts_mis = np.unique(mis_class, return_counts = True)

cnt = []
for i in labels_mis:
    cnt.append(counts[np.where(labels == i)][0])
cnt = np.array(cnt)

mis_rate = counts_mis/cnt

import pandas as pd
pd.DataFrame({'miss_classes': labels_mis, 
              'miss_counts': counts_mis,
              'total_counts': cnt,
              'miss_rate': mis_rate})

,miss_classes,miss_counts,total_counts,miss_rate
0,5,3,3,1.0
1,11,1,1,1.0
2,14,1,2,0.5
3,25,1,1,1.0
4,31,1,2,0.5
5,37,2,2,1.0
6,40,1,1,1.0
7,42,1,1,1.0
